# Project - Amazon_jobs_data

---

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
from konlpy.tag import Twitter
import lightgbm as lgb

In [3]:
jobs_df = pd.read_csv("amazon_jobs_dataset.csv")

In [4]:
jobs_df.shape

(3493, 7)

In [5]:
jobs_df = jobs_df.dropna()

In [6]:
jobs_df.shape

(3484, 7)

In [7]:
jobs_df.tail()

,Unnamed: 0,Title,location,Posting_date,DESCRIPTION,BASIC QUALIFICATIONS,PREFERRED QUALIFICATIONS
3488,3488,Software Engineer,"US, NV, Las Vegas","August 29, 2012",What are you currently doing? We’re building a...,· 5+ years production Java software developmen...,· 7+ years Java software development in a prod...
3489,3489,Software Development Engineer,"US, WA, Seattle","April 23, 2012",At Amazon Voice and Advanced Natural Shopping ...,* Bachelor’s Degree in Computer Science or rel...,* Experience building complex software systems...
3490,3490,Software Development Engineer III - Mar. BR 2012,"US, WA, Seattle","March 6, 2012",Employer: Amazon Corporate LLCPosition: Softwa...,Basic Qualifications:Education: Master’s degre...,-
3491,3491,Software Development Engineer in Test II - IMM546,"US, WA, Seattle","November 28, 2011",POSITIONS AVAILABLECompany: Amazon Corporate L...,-,-
3492,3492,Software Development Engineer in Test,"US, WA, Seattle","July 7, 2011",Kindle and Amazon are impacting the world with...,"· Bachelor’s degree in Computer Science, Compu...",· Experience with open source tools and resour...


In [8]:
cat = []
for idx, con in enumerate(jobs_df['Title'].unique()):
    cat.append(con)

In [9]:
cats = pd.DataFrame(cat , columns=["job"])

In [10]:
cats['category'] = cats.index + 1

In [11]:
A_jobs_df = pd.merge(jobs_df, cats, left_on="Title", right_on="job", how='left')

In [12]:
A_jobs_df = A_jobs_df.drop('Title', axis=1)

In [13]:
A_jobs_df.tail()

,Unnamed: 0,location,Posting_date,DESCRIPTION,BASIC QUALIFICATIONS,PREFERRED QUALIFICATIONS,job,category
3479,3488,"US, NV, Las Vegas","August 29, 2012",What are you currently doing? We’re building a...,· 5+ years production Java software developmen...,· 7+ years Java software development in a prod...,Software Engineer,132
3480,3489,"US, WA, Seattle","April 23, 2012",At Amazon Voice and Advanced Natural Shopping ...,* Bachelor’s Degree in Computer Science or rel...,* Experience building complex software systems...,Software Development Engineer,2
3481,3490,"US, WA, Seattle","March 6, 2012",Employer: Amazon Corporate LLCPosition: Softwa...,Basic Qualifications:Education: Master’s degre...,-,Software Development Engineer III - Mar. BR 2012,1554
3482,3491,"US, WA, Seattle","November 28, 2011",POSITIONS AVAILABLECompany: Amazon Corporate L...,-,-,Software Development Engineer in Test II - IMM546,1555
3483,3492,"US, WA, Seattle","July 7, 2011",Kindle and Amazon are impacting the world with...,"· Bachelor’s degree in Computer Science, Compu...",· Experience with open source tools and resour...,Software Development Engineer in Test,161


In [14]:
A_jobs_df.isnull().sum()

Unnamed: 0                  0
location                    0
Posting_date                0
DESCRIPTION                 0
BASIC QUALIFICATIONS        0
PREFERRED QUALIFICATIONS    0
job                         0
category                    0
dtype: int64

In [15]:
A_jobs_df['job'].value_counts()

Software Development Engineer                                              695
Software Development Manager                                               122
Senior Software Development Engineer                                       114
Sr. Software Development Engineer                                           58
Software Development Engineer                                               56
Software Development Engineer II                                            50
Web Development Engineer                                                    20
Software Development Engineer, Amazon Alexa                                 18
Senior Software Engineer                                                    18
Software Development Engineer II - Alexa Deep Product Knowledge             17
Software Development Engineer – In-Memory Distributed Systems               13
Sr Software Development Engineer                                            12
Software Development Engineer (Fullstack)           

In [16]:
A_jobs_df['category'].value_counts()

2       695
1       122
95      114
53       58
5        56
50       50
44       20
239      18
135      18
671      17
46       13
132      12
519      12
348      12
621      12
126      11
7        10
117      10
80        9
571       9
139       9
641       8
4         8
61        7
628       7
527       7
515       7
762       7
455       7
284       6
       ... 
485       1
487       1
489       1
495       1
499       1
501       1
503       1
511       1
449       1
443       1
395       1
441       1
397       1
399       1
401       1
403       1
405       1
407       1
409       1
411       1
415       1
417       1
419       1
421       1
423       1
425       1
429       1
435       1
437       1
1555      1
Name: category, Length: 1555, dtype: int64

In [17]:
A_jobs_df = A_jobs_df[A_jobs_df['category'].isin(['2','1','95'])]

In [18]:
A_jobs_df = A_jobs_df.drop(["Unnamed: 0", 'location', 'Posting_date'], axis=1)

In [19]:
A_jobs_df = A_jobs_df.reset_index(drop=True)

In [20]:
A_jobs_df.tail()

,DESCRIPTION,BASIC QUALIFICATIONS,PREFERRED QUALIFICATIONS,job,category
926,"You: “Alexa, I’m looking for a job where I can...",You have industry-leading technical abilities....,· Excellent design skills with a proven abilit...,Software Development Engineer,2
927,Are you seeking a startup environment where yo...,· Master's Degree in Computer Science or relat...,"· Experience in leading engineering team, coac...",Software Development Manager,1
928,Do you want to be involved in one of Amazon’s ...,"· BS in Computer Science, Computer or Electric...","· MS in Computer Science, Computer/Electrical ...",Software Development Engineer,2
929,The Digital Store Platform team (DSP) creates ...,· Bachelor’s Degree in Computer Science or rel...,· Computer Science fundamentals in algorithm d...,Software Development Engineer,2
930,At Amazon Voice and Advanced Natural Shopping ...,* Bachelor’s Degree in Computer Science or rel...,* Experience building complex software systems...,Software Development Engineer,2


---

In [21]:
X_train_D, X_test_D, y_train_D, y_test_D = train_test_split(A_jobs_df['DESCRIPTION'], A_jobs_df.category, \
                                                    test_size=0.2, random_state=0)

len(X_train_D),len(X_test_D),len(y_train_D),len(y_test_D)

(744, 187, 744, 187)

In [22]:
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(A_jobs_df['BASIC QUALIFICATIONS'], A_jobs_df.category, \
                                                    test_size=0.2, random_state=0)

len(X_train_B),len(X_test_B),len(y_train_B),len(y_test_B)

(744, 187, 744, 187)

In [23]:
X_train_P, X_test_P, y_train_P, y_test_P = train_test_split(A_jobs_df['PREFERRED QUALIFICATIONS'], A_jobs_df.category, \
                                                    test_size=0.2, random_state=0)

len(X_train_P),len(X_test_P),len(y_train_P),len(y_test_P)

(744, 187, 744, 187)

---

In [24]:
from konlpy.tag import Twitter
pos_tagger = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [25]:
clf = Pipeline([
    ("vect", TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,3), \
                             stop_words=["and", "is", "the", "this", "-", ".", "·"])), 
    ("clf", MultinomialNB(alpha=0.05))
])

---

In [26]:
model_D = clf.fit(X_train_D, y_train_D)

In [27]:
y_pred_D = model_D.predict(X_test_D)

In [28]:
confusion_matrix(y_test_D, y_pred_D)

array([[  8,  10,   0],
       [  3, 132,  10],
       [  3,  16,   5]])

In [29]:
print(classification_report(y_test_D, y_pred_D))

             precision    recall  f1-score   support

          1       0.57      0.44      0.50        18
          2       0.84      0.91      0.87       145
         95       0.33      0.21      0.26        24

avg / total       0.75      0.78      0.76       187



In [30]:
accuracy_score(y_test_D, y_pred_D)

0.7754010695187166

---

In [31]:
model_B = clf.fit(X_train_B, y_train_B)

In [32]:
y_pred_B = model_B.predict(X_test_B)

In [33]:
confusion_matrix(y_test_B, y_pred_B)

array([[ 13,   5,   0],
       [  4, 128,  13],
       [  2,  14,   8]])

In [34]:
print(classification_report(y_test_B, y_pred_B))

             precision    recall  f1-score   support

          1       0.68      0.72      0.70        18
          2       0.87      0.88      0.88       145
         95       0.38      0.33      0.36        24

avg / total       0.79      0.80      0.79       187



In [35]:
accuracy_score(y_test_B, y_pred_B)

0.7967914438502673

---

In [36]:
model_P = clf.fit(X_train_P, y_train_P)

In [37]:
y_pred_P = model_P.predict(X_test_P)

In [38]:
confusion_matrix(y_test_P, y_pred_P)

array([[  6,  12,   0],
       [  8, 128,   9],
       [  3,  14,   7]])

In [39]:
print(classification_report(y_test_P, y_pred_P))

             precision    recall  f1-score   support

          1       0.35      0.33      0.34        18
          2       0.83      0.88      0.86       145
         95       0.44      0.29      0.35        24

avg / total       0.73      0.75      0.74       187



In [40]:
accuracy_score(y_test_P, y_pred_P)

0.7540106951871658

종합 결과 필요 or Performance가 가장 좋은 feature를 선택.

---

In [41]:
vect = CountVectorizer()
vect.fit(A_jobs_df['DESCRIPTION'])
vect.vocabulary_

{'you': 5740,
 'are': 387,
 'an': 304,
 'experienced': 1939,
 'hands': 2391,
 'on': 3536,
 'manager': 3136,
 'with': 5683,
 'background': 511,
 'in': 2605,
 'developing': 1455,
 'and': 313,
 'delivering': 1379,
 'software': 4794,
 'that': 5185,
 'simplifies': 4722,
 'solutions': 4800,
 'for': 2144,
 'broad': 693,
 'set': 4652,
 'of': 3515,
 'related': 4283,
 'problems': 3977,
 'have': 2417,
 'strong': 4978,
 'technical': 5138,
 'ability': 80,
 'great': 2341,
 'communication': 1020,
 'skills': 4752,
 'know': 2905,
 'how': 2502,
 'to': 5262,
 'inspire': 2702,
 'your': 5744,
 'team': 5126,
 'achieve': 124,
 'outstanding': 3632,
 'results': 4393,
 'fast': 2023,
 'paced': 3654,
 'environment': 1820,
 'track': 5302,
 'record': 4230,
 'building': 720,
 'leading': 2970,
 'teams': 5131,
 'talented': 5106,
 'engineers': 1775,
 'enjoy': 1786,
 'working': 5703,
 'smart': 4768,
 'people': 3738,
 'short': 4690,
 'skilled': 4751,
 'leader': 2967,
 'capable': 776,
 'getting': 2288,
 'things': 5204,
 '

In [42]:
check = vect.transform(A_jobs_df['DESCRIPTION']).toarray()

In [43]:
check = pd.DataFrame(check)

In [44]:
check.tail()

,0,1,2,3,4,5,6,7,8,9,...,5744,5745,5746,5747,5748,5749,5750,5751,5752,5753
926,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
927,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
929,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
930,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
check['y'] = A_jobs_df['category']

In [46]:
check.tail()

,0,1,2,3,4,5,6,7,8,9,...,5745,5746,5747,5748,5749,5750,5751,5752,5753,y
926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
927,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
929,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
930,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [67]:
le = LabelEncoder()

y_train = le.fit_transform(check['y'])
X_train = check.drop("y",axis=1)

print("Start1")
model_1 = lgb.LGBMClassifier(nthread=3)
scores_1 = cross_val_score(estimator=model_1, X=X_train, y=y_train, cv=4, scoring='neg_log_loss')
print("log_loss: %0.2f (%0.2f)" % (scores_1.mean(), scores_1.std() * 2), '-> None')
print("End")
print(" ")

print("Start2")
model_2 = lgb.LGBMClassifier(nthread=3, reg_alpha=0.5)
scores_2 = cross_val_score(estimator=model_2, X=X_train, y=y_train, cv=4, scoring='neg_log_loss')
print("log_loss: %0.2f (%0.2f)" % (scores_2.mean(), scores_2.std() * 2), '-> reg_alpha=0.5')
print("End")
print(" ")

print("Start3")
model_3 = lgb.LGBMClassifier(nthread=3, reg_alpha=1)
scores_3 = cross_val_score(estimator=model_3, X=X_train, y=y_train, cv=4, scoring='neg_log_loss')
print("log_loss: %0.2f (%0.2f)" % (scores_3.mean(), scores_3.std() * 2), '-> reg_alpha=1')
print("End")
print(" ")

print("Start4")
model_4 = lgb.LGBMClassifier(nthread=3, reg_alpha=0.5, learning_rate=0.05, n_estimator=200)
scores_4 = cross_val_score(estimator=model_4, X=X_train, y=y_train, cv=4, scoring='neg_log_loss')
print("log_loss: %0.2f (%0.2f)" % (scores_4.mean(), scores_4.std() * 2), '-> reg_alpha=0.5, l_r=0.05, n_estimator=200')
print("End")
print(" ")

print("Start5")
model_5 = lgb.LGBMClassifier(nthread=3, reg_alpha=1, learning_rate=0.05, n_estimator=300)
scores_5 = cross_val_score(estimator=model_5, X=X_train, y=y_train, cv=4, scoring='neg_log_loss')
print("log_loss: %0.2f (%0.2f)" % (scores_5.mean(), scores_5.std() * 2), '-> reg_alpha=1, l_r=0.05, n_estimator=300')
print("End")
print(" ")

print("Trial_Error")
model_6 = lgb.LGBMClassifier(nthread=3, n_estimators=200)
scores_6 = cross_val_score(estimator=model_6, X=X_train, y=y_train, cv=4, scoring='neg_log_loss')
print("log_loss: %0.2f (%0.2f)" % (scores_6.mean(), scores_6.std() * 2), '-> n_estimator=200')
print("End")
print(" ")

Start1
log_loss: -0.64 (0.15) -> None
End
 
Start2
log_loss: -0.53 (0.11) -> reg_alpha=0.5
End
 
Start3
log_loss: -0.51 (0.11) -> reg_alpha=1
End
 
Start4
log_loss: -0.48 (0.08) -> reg_alpha=0.5, l_r=0.05, n_estimator=200
End
 
Start5
log_loss: -0.48 (0.08) -> reg_alpha=1, l_r=0.05, n_estimator=300
End
 
Trial_Error
log_loss: -0.97 (0.30) -> n_estimator=200
End
 


In [60]:
model_lgb = lgb.LGBMClassifier(nthread=3, n_estimators=200)

In [61]:
model_lgb.fit(check.drop("y",axis=1), check['y'])

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=200,
        n_jobs=-1, nthread=3, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1)

In [62]:
y_pred_l = model_lgb.predict(check.drop("y",axis=1))

In [63]:
confusion_matrix(check['y'], y_pred_l)

array([[121,   1,   0],
       [  1, 689,   5],
       [  0,  14, 100]])

In [64]:
print(classification_report(check['y'], y_pred_l))

             precision    recall  f1-score   support

          1       0.99      0.99      0.99       122
          2       0.98      0.99      0.98       695
         95       0.95      0.88      0.91       114

avg / total       0.98      0.98      0.98       931



In [65]:
accuracy_score(check['y'], y_pred_l)

0.9774436090225563

다른 columns 추가 분석 필요

In [ ]:
vect = CountVectorizer()
vect.fit(A_jobs_df[''])
vect.vocabulary_